In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import functools as F

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
#from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Imputer as SimpleImputer
from sklearn import metrics   #Additional scklearn functions
#from sklearn.grid_search import GridSearchCV #Perforing grid search
import xgboost as xgb
import shap
#Bayesian optimization for model parameters
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


# Any results you write to the current directory are saved as output.

**Standard Util Functions**

In [ ]:
def dataImputation(orig_df):
    df = orig_df.copy()
    # make new columns indicating what will be imputed
    cols_with_missing = (col for col in df.columns if df[col].isnull().any())
    #for col in cols_with_missing:
    #    df[col + '_was_missing'] = df[col].isnull()
    # Imputation
    my_imputer = SimpleImputer()
    df = pd.DataFrame(my_imputer.fit_transform(df),columns=df.columns.values)
    
    
    print(df.columns)
    #print(orig_df.columns)
    return df

#dataImputation(application_train)


In [ ]:
#Data Input

application_train = pd.read_csv('../input/application_train.csv')
POS_CASH_balance = pd.read_csv('../input/POS_CASH_balance.csv')
bureau_balance = pd.read_csv('../input/bureau_balance.csv')
previous_application = pd.read_csv('../input/previous_application.csv')
installments_payments = pd.read_csv('../input/installments_payments.csv')
credit_card_balance = pd.read_csv('../input/credit_card_balance.csv')
bureau = pd.read_csv('../input/bureau.csv')
application_test = pd.read_csv('../input/application_test.csv')

**Below Code is finding posterior prob given priro. But Taken count. Next Amount sholud also be used to find posterior mean**

In [ ]:
res = pd.merge(application_train,bureau,on=['SK_ID_CURR'],how='inner')
count = res.groupby(by = ['CREDIT_TYPE','TARGET']).size().reset_index(name='cnt')
#count
Total_Population = res.groupby(by = ['TARGET']).size().reset_index(name = 'sample_size')
total = Total_Population['sample_size'].sum()
Total_Population['t_prop'] = Total_Population['sample_size']/total
#
Credit_Population = res.groupby(by = ['CREDIT_TYPE']).size().reset_index(name = 'csample_size')
Credit_Population['c_prop'] = Credit_Population['csample_size']/total
#
Credit_Type_Population = count.groupby(by = ['CREDIT_TYPE'])['cnt'].sum().to_dict()
count['p_prop'] = count.apply(lambda x,y: x['cnt'] /y[x['CREDIT_TYPE']],args=(Credit_Type_Population,),axis=1 )
count["prop_p_t"] = (count['p_prop'] * Total_Population[Total_Population['TARGET']==1]['t_prop'].iloc[0])
count = pd.merge(count,Credit_Population,on =['CREDIT_TYPE'],how='left')
#Credit_Type_Population['Car loan']
count['post_prop'] = count['prop_p_t'] / count['c_prop']
count = count[count['TARGET'] ==1]
count
#Findind post_prop of amt
amt_credit = res.groupby(by = ['CREDIT_TYPE','TARGET'])['AMT_CREDIT'].sum().reset_index(name='amt')
#count
#-- instead of size do sum of amt
Total_Population_amt = res.groupby(by = ['TARGET'])['AMT_CREDIT'].sum().reset_index(name = 'sample_size')
total_amt = Total_Population_amt['sample_size'].sum()
Total_Population_amt['t_prop_amt'] = Total_Population_amt['sample_size']/total_amt
#
Credit_Population_amt = res.groupby(by = ['CREDIT_TYPE'])['AMT_CREDIT'].sum().reset_index(name = 'csample_size_amt')
Credit_Population_amt['c_prop_amt'] = Credit_Population_amt['csample_size_amt']/total_amt
#
Credit_Type_Population_amt = amt_credit.groupby(by = ['CREDIT_TYPE'])['amt'].sum().to_dict()
amt_credit['p_prop_amt'] = amt_credit.apply(lambda x,y: x['amt'] /y[x['CREDIT_TYPE']],args=(Credit_Type_Population_amt,),axis=1 )
amt_credit["prop_p_t_amt"] = (amt_credit['p_prop_amt'] * Total_Population_amt[Total_Population_amt['TARGET']==1]['t_prop_amt'].iloc[0])
amt_credit = pd.merge(amt_credit,Credit_Population_amt,on =['CREDIT_TYPE'],how='left')
#Credit_Type_Population['Car loan']
amt_credit['post_prop_amt'] = amt_credit['prop_p_t_amt'] / amt_credit['c_prop_amt']
amt_credit = amt_credit[amt_credit['TARGET'] ==1]
amt_credit

**Application Train Features**

In [ ]:
#train_final = dataImputation(application_train)
#train_final

application_train['NEW_CREDIT_TO_ANNUITY_RATIO'] = application_train['AMT_CREDIT'] / application_train['AMT_ANNUITY']
application_train['NEW_CREDIT_TO_GOODS_RATIO'] = application_train['AMT_CREDIT'] / application_train['AMT_GOODS_PRICE']
application_train['ANNUITY_TO_INCOME'] = application_train['AMT_ANNUITY'] / application_train['AMT_INCOME_TOTAL']

#application_train['NEW_DOC_IND_KURT'] = application_train[docs].kurtosis(axis=1)
#application_train['NEW_LIVE_IND_SUM'] = application_train[live].sum(axis=1)

# family features
application_train['NEW_INC_PER_CHLD'] = application_train['AMT_INCOME_TOTAL'] / (1 + application_train['CNT_CHILDREN'])
application_train['NEW_INCOME_PER_PERS'] = application_train['AMT_INCOME_TOTAL'] / application_train['CNT_FAM_MEMBERS']
application_train['CHILDREN_RATIO'] = application_train['CNT_CHILDREN'] / application_train['CNT_FAM_MEMBERS']
application_train['CNT_NO_CHILD'] = application_train['CNT_FAM_MEMBERS'] - application_train['CNT_CHILDREN']
application_train['CREDIT_PER_PERS'] = application_train['AMT_CREDIT'] / application_train['CNT_FAM_MEMBERS']
application_train['CREDIT_PER_CHILD'] = application_train['AMT_CREDIT'] / (1 + application_train['CNT_CHILDREN'])
application_train['CREDIT_PER_NO_CHILD'] = application_train['AMT_CREDIT'] / application_train['CNT_NO_CHILD']

# extract most features from external sources
application_train['NEW_SOURCES_PROD'] = application_train['EXT_SOURCE_1'] * application_train['EXT_SOURCE_2'] * application_train['EXT_SOURCE_3']
application_train['NEW_EXT_SOURCES_MEAN'] = application_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
application_train['NEW_SCORES_STD'] = application_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
application_train['NEW_SCORES_STD'] = application_train['NEW_SCORES_STD'].fillna(application_train['NEW_SCORES_STD'].mean())

application_train['NEW_CAR_TO_BIRTH_RATIO'] = application_train['OWN_CAR_AGE'] / application_train['DAYS_BIRTH']
application_train['NEW_CAR_TO_EMPLOY_RATIO'] = application_train['OWN_CAR_AGE'] / application_train['DAYS_EMPLOYED']
application_train['NEW_PHONE_TO_BIRTH_RATIO'] = application_train['DAYS_LAST_PHONE_CHANGE'] / application_train['DAYS_BIRTH']
application_train['NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER'] = application_train['DAYS_LAST_PHONE_CHANGE'] / application_train['DAYS_EMPLOYED']
application_train['NEW_CREDIT_TO_INCOME_RATIO'] = application_train['AMT_CREDIT'] / application_train['AMT_INCOME_TOTAL']
#New Features
application_train['INCOME_BY_DAYS_EMP'] = application_train['AMT_INCOME_TOTAL']/(1+application_train['DAYS_EMPLOYED'])
application_train['LOW_OCC'] = map(lambda x: 1 if x.isin(['Cooking staff','Drivers','Laborers','Low-skill Laborers','Security staff','Waiters/barmen staff']) else 0,application_train['OCCUPATION_TYPE'])
application_train['AVG_EXT_SOURCE'] = (application_train['EXT_SOURCE_1'] + 2*application_train['EXT_SOURCE_2'] + 3*application_train['EXT_SOURCE_3']) / 3
# Below features to test Feature importance
application_train['DAYS_EMP_TO_BIRTH_RATIO'] = (application_train['DAYS_EMPLOYED']/application_train['DAYS_BIRTH'])
application_train['AMT_ANNUITY_TO_DAYS_EMP_RATIO'] = application_train['AMT_ANNUITY'] / application_train['DAYS_EMPLOYED']
application_train['AMT_ANNUITY_TO_DAYS_BIRTH_RATIO'] = application_train['AMT_ANNUITY'] / application_train['DAYS_BIRTH']

application_train['AMT_INC_TO_DAYS_BIRTH_RATIO'] = application_train['AMT_INCOME_TOTAL'] / application_train['DAYS_BIRTH']
application_train['AMT_GOODS_PER_PERS'] = application_train['AMT_GOODS_PRICE'] / application_train['CNT_FAM_MEMBERS']
application_train['AMT_ANNUITY_PER_PERS'] = application_train['AMT_ANNUITY'] / application_train['CNT_FAM_MEMBERS']
application_train['INCOME_BY_AVG_EXT_SOURCE'] = application_train['AMT_INCOME_TOTAL']/(1+application_train['AVG_EXT_SOURCE'])


**Application Test Features**

In [ ]:
#train_final = dataImputation(application_test)
#train_final

application_test['NEW_CREDIT_TO_ANNUITY_RATIO'] = application_test['AMT_CREDIT'] / application_test['AMT_ANNUITY']
application_test['NEW_CREDIT_TO_GOODS_RATIO'] = application_test['AMT_CREDIT'] / application_test['AMT_GOODS_PRICE']
application_test['ANNUITY_TO_INCOME'] = application_test['AMT_ANNUITY'] / application_test['AMT_INCOME_TOTAL']

#application_test['NEW_DOC_IND_KURT'] = application_test[docs].kurtosis(axis=1)
#application_test['NEW_LIVE_IND_SUM'] = application_test[live].sum(axis=1)

# family features
application_test['NEW_INC_PER_CHLD'] = application_test['AMT_INCOME_TOTAL'] / (1 + application_test['CNT_CHILDREN'])
application_test['NEW_INCOME_PER_PERS'] = application_test['AMT_INCOME_TOTAL'] / application_test['CNT_FAM_MEMBERS']
application_test['CHILDREN_RATIO'] = application_test['CNT_CHILDREN'] / application_test['CNT_FAM_MEMBERS']
application_test['CNT_NO_CHILD'] = application_test['CNT_FAM_MEMBERS'] - application_test['CNT_CHILDREN']
application_test['CREDIT_PER_PERS'] = application_test['AMT_CREDIT'] / application_test['CNT_FAM_MEMBERS']
application_test['CREDIT_PER_CHILD'] = application_test['AMT_CREDIT'] / (1 + application_test['CNT_CHILDREN'])
application_test['CREDIT_PER_NO_CHILD'] = application_test['AMT_CREDIT'] / application_test['CNT_NO_CHILD']

# extract most features from external sources
application_test['NEW_SOURCES_PROD'] = application_test['EXT_SOURCE_1'] * application_test['EXT_SOURCE_2'] * application_test['EXT_SOURCE_3']
application_test['NEW_EXT_SOURCES_MEAN'] = application_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
application_test['NEW_SCORES_STD'] = application_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
application_test['NEW_SCORES_STD'] = application_test['NEW_SCORES_STD'].fillna(application_test['NEW_SCORES_STD'].mean())

application_test['NEW_CAR_TO_BIRTH_RATIO'] = application_test['OWN_CAR_AGE'] / application_test['DAYS_BIRTH']
application_test['NEW_CAR_TO_EMPLOY_RATIO'] = application_test['OWN_CAR_AGE'] / application_test['DAYS_EMPLOYED']
application_test['NEW_PHONE_TO_BIRTH_RATIO'] = application_test['DAYS_LAST_PHONE_CHANGE'] / application_test['DAYS_BIRTH']
application_test['NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER'] = application_test['DAYS_LAST_PHONE_CHANGE'] / application_test['DAYS_EMPLOYED']
application_test['NEW_CREDIT_TO_INCOME_RATIO'] = application_test['AMT_CREDIT'] / application_test['AMT_INCOME_TOTAL']
# New Features
application_test['INCOME_BY_DAYS_EMP'] = application_test['AMT_INCOME_TOTAL']/(1+application_test['DAYS_EMPLOYED'])
application_test['LOW_OCC'] = map(lambda x: 1 if x.isin(['Cooking staff','Drivers','Laborers','Low-skill Laborers','Security staff','Waiters/barmen staff']) else 0,application_test['OCCUPATION_TYPE'])
application_test['AVG_EXT_SOURCE'] =(application_test['EXT_SOURCE_1'] + 2*application_test['EXT_SOURCE_2'] + 3*application_test['EXT_SOURCE_3']) / 3
# Below features to test Feature importance
application_test['DAYS_EMP_TO_BIRTH_RATIO'] = (application_test['DAYS_EMPLOYED']/application_test['DAYS_BIRTH'])
application_test['AMT_ANNUITY_TO_DAYS_EMP_RATIO'] = application_test['AMT_ANNUITY'] / application_test['DAYS_EMPLOYED']
application_test['AMT_ANNUITY_TO_DAYS_BIRTH_RATIO'] = application_test['AMT_ANNUITY'] / application_test['DAYS_BIRTH']

application_test['AMT_INC_TO_DAYS_BIRTH_RATIO'] = application_test['AMT_INCOME_TOTAL'] / application_test['DAYS_BIRTH']
application_test['AMT_GOODS_PER_PERS'] = application_test['AMT_GOODS_PRICE'] / application_test['CNT_FAM_MEMBERS']
application_test['AMT_ANNUITY_PER_PERS'] = application_test['AMT_ANNUITY'] / application_test['CNT_FAM_MEMBERS']
application_test['INCOME_BY_AVG_EXT_SOURCE'] = application_test['AMT_INCOME_TOTAL']/(1+application_test['AVG_EXT_SOURCE'])

**BUREAU BASED FEATURES**

In [ ]:
BUREAU_FEATURES=bureau[['SK_ID_CURR']].drop_duplicates()
#FEATURE 1 - NUMBER OF PAST LOANS PER CUSTOMER
BUREAU_LOAN_COUNT = bureau[['SK_ID_CURR','CREDIT_TYPE']].groupby(by=['SK_ID_CURR'])['CREDIT_TYPE'].count().reset_index(name='BUREAU_LOAN_COUNT')
BUREAU_FEATURES = BUREAU_FEATURES.merge(BUREAU_LOAN_COUNT,on=['SK_ID_CURR'],how='left')

#FEATURE 2 - NUMBER OF TYPES OF PAST LOANS PER CUSTOMER
BUREAU_LOAN_TYPES = bureau[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})
BUREAU_FEATURES = BUREAU_FEATURES.merge(BUREAU_LOAN_TYPES,on=['SK_ID_CURR'],how='left')

#Feature 3 - Net Sum of Posterior Probabilites
BUREAU_NET_SUM_POST_PROP = pd.merge(bureau[['SK_ID_CURR', 'CREDIT_TYPE']],count[['CREDIT_TYPE','post_prop']] ,on=['CREDIT_TYPE'],how='left' ).groupby(by = ['SK_ID_CURR'])['post_prop'].sum().reset_index(name='BUREAU_NET_SUM_POST_PROP')
BUREAU_FEATURES = BUREAU_FEATURES.merge(BUREAU_NET_SUM_POST_PROP,on=['SK_ID_CURR'],how='left')

#Feature 4 - Net Product of Posterior Probabilites
BUREAU_NET_PROD_POST_PROP = pd.merge(bureau[['SK_ID_CURR', 'CREDIT_TYPE']],count[['CREDIT_TYPE','post_prop']] ,on=['CREDIT_TYPE'],how='left' ).groupby(by = ['SK_ID_CURR'])['post_prop'].prod().reset_index(name='BUREAU_NET_PROD_POST_PROP')
BUREAU_FEATURES = BUREAU_FEATURES.merge(BUREAU_NET_PROD_POST_PROP,on=['SK_ID_CURR'],how='left')

#Feature 5 - Net Sum of AMT Posterior Probabilites
BUREAU_NET_SUM_AMT_POST_PROP = pd.merge(bureau[['SK_ID_CURR', 'CREDIT_TYPE']],amt_credit[['CREDIT_TYPE','post_prop_amt']] ,on=['CREDIT_TYPE'],how='left' ).groupby(by = ['SK_ID_CURR'])['post_prop_amt'].sum().reset_index(name='BUREAU_NET_SUM_AMT_POST_PROP')
BUREAU_FEATURES = BUREAU_FEATURES.merge(BUREAU_NET_SUM_AMT_POST_PROP,on=['SK_ID_CURR'],how='left')

#Feature 6 - Net Product of AMT Posterior Probabilites
BUREAU_NET_PROD_AMT_POST_PROP = pd.merge(bureau[['SK_ID_CURR', 'CREDIT_TYPE']],amt_credit[['CREDIT_TYPE','post_prop_amt']] ,on=['CREDIT_TYPE'],how='left' ).groupby(by = ['SK_ID_CURR'])['post_prop_amt'].prod().reset_index(name='BUREAU_NET_PROD_AMT_POST_PROP')
BUREAU_FEATURES = BUREAU_FEATURES.merge(BUREAU_NET_PROD_AMT_POST_PROP,on=['SK_ID_CURR'],how='left')

#Ratio BUREAU_LOAN_TYPES/BUREAU_LOAN_COUNT
    #Is the Customer diversified in taking multiple types of Loan or Focused on a single type of loan
BUREAU_FEATURES['BUREAU_LOAN_PER_TYPE'] = BUREAU_FEATURES['BUREAU_LOAN_COUNT'] / BUREAU_FEATURES['BUREAU_LOAN_TYPES']
#BUREAU_FEATURES

DAY_CREDIT_DIFF=bureau[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR']).apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)
DAY_CREDIT_DIFF['DAYS_CREDIT1'] = DAY_CREDIT_DIFF['DAYS_CREDIT']*-1
DAY_CREDIT_DIFF['DAYS_DIFF'] = DAY_CREDIT_DIFF.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT1'].diff().fillna(0)
# Now find Mean of above DAY_CREDIT_DIFF
DAYS_DIFF_MEAN = DAY_CREDIT_DIFF.groupby(by = ['SK_ID_CURR'])['DAYS_DIFF'].mean().reset_index(name='DAYS_DIFF_MEAN').fillna(0)
# Now find standarad deviation of above DAY_CREDIT_DIFF
DAYS_DIFF_STD = DAY_CREDIT_DIFF.groupby(by=['SK_ID_CURR'])['DAYS_DIFF'].std().reset_index(name='DAYS_DIFF_STD').fillna(1)
#DAYS_DIFF_RATIO = DAY_CREDIT_DIFF['DAYS_DIFF_MEAN']/DAY_CREDIT_DIFF['DAYS_DIFF_STD']
DAYS_DIFF_JOIN = DAYS_DIFF_MEAN.merge(DAYS_DIFF_STD,on='SK_ID_CURR',how='inner')
DAYS_DIFF_JOIN['DAYS_DIFF_RATIO'] = DAYS_DIFF_JOIN['DAYS_DIFF_MEAN']/DAYS_DIFF_JOIN['DAYS_DIFF_STD']
#DAYS_DIFF_JOIN
#DAY_CREDIT_DIFF

BUREAU_FEATURES = BUREAU_FEATURES.merge(DAYS_DIFF_JOIN,on='SK_ID_CURR',how='left')
bureau['BUREAU_DAY_CREDIT_BY_CNT_PROLONG']=(bureau['DAYS_CREDIT'] *-1)/(bureau['CNT_CREDIT_PROLONG']+1)
BUREAU_CNT_PROLONG_FEATURES = bureau[bureau['CREDIT_ACTIVE']=="Active"].groupby(by = ['SK_ID_CURR'])[
    'BUREAU_DAY_CREDIT_BY_CNT_PROLONG' ,
    'CNT_CREDIT_PROLONG'
].agg(['min','max','sum','mean','var','count','std']).reset_index()

BUREAU_CNT_PROLONG_FEATURES.columns = ['_'.join(col).strip('_') for col in BUREAU_CNT_PROLONG_FEATURES.columns.values]
BUREAU_FEATURES = BUREAU_FEATURES.merge(BUREAU_CNT_PROLONG_FEATURES,on='SK_ID_CURR',how='left')

#bureau.head()

In [ ]:
# Credit card data - numerical features
wm = lambda x: np.average(x, weights=-1/credit_card_balance.loc[x.index, 'MONTHS_BALANCE'])
credit_card_avgs = credit_card_balance.groupby('SK_ID_CURR').agg(wm)


**Previous_application Past Application in HomeCredit (Current Application ID x Past Application ID)**

In [ ]:
# Percentage of  Past Applied Credit Rejected
#previous_application['PCT_PAST_REJECT']= previous_application[['SK_ID_CURR','NAME_CONTRACT_STATUS']].groupby(by=['SK_ID_CURR']).agg(np.sum(np.where(previous_application[['NAME_CONTRACT_STATUS']]=="Refused")).astype('float'))
PREV_APP_FEATURES = previous_application['SK_ID_CURR'].drop_duplicates()

#Feature Previous Refused application AMT_CREDIT/AMT_APPLICATION
PREV_REFUSED_RATIO_CREDIT_APPLICATION = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') ].groupby(by=['SK_ID_CURR']).apply(lambda x: x['AMT_CREDIT'].sum()/x['AMT_APPLICATION'].sum()).reset_index(name='PREV_REFUSED_RATIO_CREDIT_APPLICATION')

#Feature Previous Refused application AMT_CREDIT/AMT_APPLICATION
PREV_NOT_REFUSED_RATIO_CREDIT_APPLICATION = previous_application[(previous_application['NAME_CONTRACT_STATUS'] != 'Refused') ].groupby(by=['SK_ID_CURR']).apply(lambda x: x['AMT_CREDIT'].sum()/x['AMT_APPLICATION'].sum()).reset_index(name='PREV_NOT_REFUSED_RATIO_CREDIT_APPLICATION')

#Feature Previous Refused application AMT_ANNUITY/AMT_APPLICATION
PREV_REFUSED_RATIO_ANNUITY_APPLICATION = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') ].groupby(by=['SK_ID_CURR']).apply(lambda x: x['AMT_ANNUITY'].sum()/x['AMT_APPLICATION'].sum()).reset_index(name='PREV_REFUSED_RATIO_ANNUITY_APPLICATION')

#Feature Previous Not Refused application AMT_ANNUITY/AMT_APPLICATION
PREV_NOT_REFUSED_RATIO_ANNUITY_APPLICATION = previous_application[(previous_application['NAME_CONTRACT_STATUS'] != 'Refused') ].groupby(by=['SK_ID_CURR']).apply(lambda x: x['AMT_ANNUITY'].sum()/x['AMT_APPLICATION'].sum()).reset_index(name='PREV_NOT_REFUSED_RATIO_ANNUITY_APPLICATION')

PREV_NFLAG_INSURED_SUM = previous_application.groupby(['SK_ID_CURR'])['NFLAG_INSURED_ON_APPROVAL'].sum().reset_index(name='PREV_NFLAG_INSURED_SUM')

# Feature Previous Refused application AMT_GOODS_PRICE/AMT_CREDIT
PREV_NOT_REFUSED_RATIO_AMT_GOODS_PRICE_CREDIT = previous_application[(previous_application['NAME_CONTRACT_STATUS'] != 'Refused') ].groupby(by=['SK_ID_CURR']).apply(lambda x: x['AMT_GOODS_PRICE'].sum()/x['AMT_CREDIT'].sum()).reset_index(name='PREV_NOT_REFUSED_RATIO_AMT_GOODS_PRICE_CREDIT')

# Feature Previous Refused application AMT_GOODS_PRICE/AMT_CREDIT
PREV_REFUSED_RATIO_AMT_GOODS_PRICE_CREDIT = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') ].groupby(by=['SK_ID_CURR']).apply(lambda x: x['AMT_GOODS_PRICE'].sum()/x['AMT_CREDIT'].sum()).reset_index(name='PREV_REFUSED_RATIO_AMT_GOODS_PRICE_CREDIT')

#Feature Previous NOT Refused application AMT_GOODS_PRICE/AMT_ANNUITY
PREV_NOT_REFUSED_RATIO_GOODS_ANNUITY = previous_application[(previous_application['NAME_CONTRACT_STATUS'] != 'Refused') ].groupby(by=['SK_ID_CURR']).apply(lambda x: x['AMT_GOODS_PRICE'].sum()/x['AMT_ANNUITY'].sum()).reset_index(name='PREV_NOT_REFUSED_RATIO_GOODS_ANNUITY')

#Feature Previous Refused application AMT_GOODS_PRICE/AMT_ANNUITY
PREV_REFUSED_RATIO_GOODS_ANNUITY = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') ].groupby(by=['SK_ID_CURR']).apply(lambda x: x['AMT_GOODS_PRICE'].sum()/x['AMT_ANNUITY'].sum()).reset_index(name='PREV_REFUSED_RATIO_GOODS_ANNUITY')

# Feature NAME_PAYMENT_TYPE = 'Cash through the bank'. Find Ratio AMT_GOODS_PRICE/AMT_ANNUITY

#NFLAG_INSURED_ON_APPROVAL
#NFLAG_MICRO_CASH

'''
These Features are yet to be developed
    PREV_REJECTED_TOTAL_DEBT_TO_INCOME_RATIO
    PREV_REJECTED_AMT_CREDIT_TO_INCOME_RATIO
'''
previous_application['PREV_CASH_TOTAL_DEBT_TO_AMT_CREDIT'] = (previous_application[previous_application['NAME_CONTRACT_TYPE'] =='Cash loans']['AMT_ANNUITY'] * previous_application[previous_application['NAME_CONTRACT_TYPE'] =='Cash loans']['CNT_PAYMENT']) / previous_application[previous_application['NAME_CONTRACT_TYPE'] =='Cash loans']['AMT_CREDIT']

previous_application['PREV_NON_CASH_TOTAL_DEBT_TO_AMT_CREDIT'] = (previous_application[previous_application['NAME_CONTRACT_TYPE'] !='Cash loans']['AMT_ANNUITY'] * previous_application[previous_application['NAME_CONTRACT_TYPE'] !='Cash loans']['CNT_PAYMENT']) / previous_application[previous_application['NAME_CONTRACT_TYPE'] !='Cash loans']['AMT_CREDIT']

previous_application_features = previous_application.groupby(by = ['SK_ID_CURR'])['PREV_NON_CASH_TOTAL_DEBT_TO_AMT_CREDIT',
                   'PREV_CASH_TOTAL_DEBT_TO_AMT_CREDIT',
                   'DAYS_DECISION'
                   ].agg(['min','max','sum','mean','var','count','std']).reset_index()
   
previous_application_features.columns = ['_'.join(col).strip('_') for col in previous_application_features.columns.values]
previous_application_features



**Installment Based Features**

In [ ]:
# aggs = ['min','max','sum','mean','var','count']
# ip = installments_payments.groupby(by = ['SK_ID_CURR','SK_ID_PREV'])['NUM_INSTALMENT_NUMBER'].agg(['count','max']).reset_index()
# ip.where((ip['max']-ip['count']) > 0)

#ip[(ip['max']-ip['count']) > 0]
#application_train[application_train['SK_ID_CURR']==100033]
#application_train[application_train['TARGET']==1].count()
#installments_payments[installments_payments['SK_ID_PREV']==2037234]
def day_distribution(x):
    from math import exp
    if (x['DAYS_INSTALMENT_DAYS_ENTRY_PAYMENT_DIFF']>=1):
        return x['AMT_PAYMENT'] * exp(-abs(float(x['DAYS_INSTALMENT_DAYS_ENTRY_PAYMENT_DIFF']))/20)
    else :
        return x['AMT_PAYMENT']
        
installments_payments['DAYS_INSTALMENT_DAYS_ENTRY_PAYMENT_DIFF'] = installments_payments['DAYS_INSTALMENT']-installments_payments['DAYS_ENTRY_PAYMENT']
installments_payments['AMT_INSTALMENT_AMT_PAYMENT_DIFF'] = installments_payments['AMT_INSTALMENT'] -installments_payments['AMT_PAYMENT']
installments_payments['AMT_PAYMENT_DECAY']  = installments_payments[['DAYS_INSTALMENT_DAYS_ENTRY_PAYMENT_DIFF','AMT_PAYMENT']].apply(day_distribution,axis = 1)
installments_payments['AMT_INSTALMENT_AMT_PAYMENT_DECAY_DIFF'] = installments_payments['AMT_INSTALMENT'] -installments_payments['AMT_PAYMENT_DECAY']
'''
    Also find number of miss installments
    Sum of Total Instalments per SK_ID_CURR,SK_ID_PREV
'''
installments_payments_features = installments_payments.groupby(by = ['SK_ID_CURR'])['AMT_INSTALMENT',
                   'AMT_PAYMENT',
                   'AMT_INSTALMENT_AMT_PAYMENT_DECAY_DIFF',
                   'AMT_PAYMENT_DECAY',
                   'DAYS_ENTRY_PAYMENT',
                   'DAYS_INSTALMENT',
                   'NUM_INSTALMENT_NUMBER',
                   'NUM_INSTALMENT_VERSION',
                   'DAYS_INSTALMENT_DAYS_ENTRY_PAYMENT_DIFF',
                   'AMT_INSTALMENT_AMT_PAYMENT_DIFF'
                   ].agg(['min','max','sum','mean','var','count','std']).reset_index()
installments_payments_features.columns = ['_'.join(col).strip('_') for col in installments_payments_features.columns.values]
installments_payments_features.head(5)

**Finally Joining All the Features**

In [ ]:
Join_DF_LIST = [application_train,BUREAU_FEATURES,PREV_REFUSED_RATIO_CREDIT_APPLICATION,PREV_NOT_REFUSED_RATIO_CREDIT_APPLICATION,PREV_REFUSED_RATIO_ANNUITY_APPLICATION,PREV_NOT_REFUSED_RATIO_ANNUITY_APPLICATION,PREV_NFLAG_INSURED_SUM,PREV_NOT_REFUSED_RATIO_AMT_GOODS_PRICE_CREDIT,PREV_REFUSED_RATIO_AMT_GOODS_PRICE_CREDIT,PREV_NOT_REFUSED_RATIO_GOODS_ANNUITY,PREV_REFUSED_RATIO_GOODS_ANNUITY,installments_payments_features,previous_application_features]
application_train = F.reduce(lambda left,right:pd.merge(left,right,on='SK_ID_CURR',how='left'), Join_DF_LIST)


Test_Join_DF_LIST = [application_test,BUREAU_FEATURES,PREV_REFUSED_RATIO_CREDIT_APPLICATION,PREV_NOT_REFUSED_RATIO_CREDIT_APPLICATION,PREV_REFUSED_RATIO_ANNUITY_APPLICATION,PREV_NOT_REFUSED_RATIO_ANNUITY_APPLICATION,PREV_NFLAG_INSURED_SUM,PREV_NOT_REFUSED_RATIO_AMT_GOODS_PRICE_CREDIT,PREV_REFUSED_RATIO_AMT_GOODS_PRICE_CREDIT,PREV_NOT_REFUSED_RATIO_GOODS_ANNUITY,PREV_REFUSED_RATIO_GOODS_ANNUITY,installments_payments_features,previous_application_features]
application_test = F.reduce(lambda left,right:pd.merge(left,right,on='SK_ID_CURR',how='left'), Test_Join_DF_LIST)


**Preprocessing Train Data**

In [ ]:
#previous_application.columns.values
#pd.merge(application_train,previous_application,on="SK_ID_CURR")
#previous_application[previous_application["SK_ID_CURR"] == 100002]
#bureau

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
le_at= LabelEncoder()
application_train_colummns = application_train.select_dtypes(include= ['object']).columns.values
application_train[application_train_colummns] = application_train[application_train_colummns].astype(str).apply(le_at.fit_transform)

#Filling NAN's for float with mean 
#application_train.head(10)
application_train_float_columns =application_train.select_dtypes(include= ['float64']).columns.values
application_train[application_train_float_columns]=application_train[application_train_float_columns].fillna(application_train[application_train_float_columns].mean())

application_train = application_train.replace([np.inf, -np.inf], 0)


**Preprocessing Test Data**

In [ ]:
application_test_colummns = application_test.select_dtypes(include= ['object']).columns.values
application_test[application_test_colummns] = application_test[application_test_colummns].astype(str).apply(le_at.fit_transform)

#Filling NAN's for float with mean 
#application_test.head(10)
application_test_float_columns =application_test.select_dtypes(include= ['float64']).columns.values
application_test[application_test_float_columns]=application_test[application_test_float_columns].fillna(application_test[application_test_float_columns].mean())

application_test = application_test.replace([np.inf, -np.inf], 0)
test_final = dataImputation(application_test)

target="TARGET"
#Imputing Values using sklearn imputer
train_final = dataImputation(application_train)
#predictors = [x for x in train_final.columns.values if x not in [target,'SK_ID_CURR']]
'''Below Feature are selected from Feature selection module, 
if want to train on whole set of features then uncomment 1st line'''
#predictors = [x for x in train_final.columns.values if x not in [target,'SK_ID_CURR']]
predictors = ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'OCCUPATION_TYPE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'LIVINGAREA_AVG', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BEGINEXPLUATATION_MEDI', 'TOTALAREA_MODE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_3', 'NEW_CREDIT_TO_ANNUITY_RATIO', 'NEW_CREDIT_TO_GOODS_RATIO', 'ANNUITY_TO_INCOME', 'CREDIT_PER_PERS', 'CREDIT_PER_CHILD', 'CREDIT_PER_NO_CHILD', 'NEW_SOURCES_PROD', 'NEW_EXT_SOURCES_MEAN', 'NEW_SCORES_STD', 'NEW_CAR_TO_BIRTH_RATIO', 'NEW_CAR_TO_EMPLOY_RATIO', 'NEW_PHONE_TO_BIRTH_RATIO', 'NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER', 'NEW_CREDIT_TO_INCOME_RATIO', 'INCOME_BY_DAYS_EMP', 'AVG_EXT_SOURCE', 'DAYS_EMP_TO_BIRTH_RATIO', 'AMT_ANNUITY_TO_DAYS_EMP_RATIO', 'AMT_ANNUITY_TO_DAYS_BIRTH_RATIO', 'AMT_INC_TO_DAYS_BIRTH_RATIO', 'AMT_GOODS_PER_PERS', 'AMT_ANNUITY_PER_PERS', 'BUREAU_NET_SUM_POST_PROP', 'BUREAU_NET_PROD_POST_PROP', 'BUREAU_NET_SUM_AMT_POST_PROP', 'BUREAU_NET_PROD_AMT_POST_PROP', 'DAYS_DIFF_MEAN', 'BUREAU_DAY_CREDIT_BY_CNT_PROLONG_min', 'BUREAU_DAY_CREDIT_BY_CNT_PROLONG_max', 'BUREAU_DAY_CREDIT_BY_CNT_PROLONG_mean', 'BUREAU_DAY_CREDIT_BY_CNT_PROLONG_count', 'CNT_CREDIT_PROLONG_count', 'PREV_REFUSED_RATIO_CREDIT_APPLICATION', 'PREV_NOT_REFUSED_RATIO_CREDIT_APPLICATION', 'PREV_REFUSED_RATIO_ANNUITY_APPLICATION', 'PREV_NOT_REFUSED_RATIO_ANNUITY_APPLICATION', 'PREV_NOT_REFUSED_RATIO_AMT_GOODS_PRICE_CREDIT', 'PREV_REFUSED_RATIO_AMT_GOODS_PRICE_CREDIT', 'PREV_NOT_REFUSED_RATIO_GOODS_ANNUITY', 'PREV_REFUSED_RATIO_GOODS_ANNUITY', 'AMT_INSTALMENT_min', 'AMT_INSTALMENT_max', 'AMT_INSTALMENT_sum', 'AMT_INSTALMENT_mean', 'AMT_PAYMENT_min', 'AMT_PAYMENT_max', 'AMT_PAYMENT_sum', 'AMT_PAYMENT_mean', 'AMT_INSTALMENT_AMT_PAYMENT_DECAY_DIFF_min', 'AMT_INSTALMENT_AMT_PAYMENT_DECAY_DIFF_max', 'AMT_INSTALMENT_AMT_PAYMENT_DECAY_DIFF_sum', 'AMT_INSTALMENT_AMT_PAYMENT_DECAY_DIFF_mean', 'AMT_PAYMENT_DECAY_min', 'AMT_PAYMENT_DECAY_sum', 'DAYS_ENTRY_PAYMENT_min', 'DAYS_ENTRY_PAYMENT_max', 'DAYS_ENTRY_PAYMENT_sum', 'DAYS_ENTRY_PAYMENT_mean', 'DAYS_ENTRY_PAYMENT_var', 'DAYS_ENTRY_PAYMENT_std', 'DAYS_INSTALMENT_min', 'DAYS_INSTALMENT_max', 'DAYS_INSTALMENT_sum', 'DAYS_INSTALMENT_mean', 'DAYS_INSTALMENT_var', 'DAYS_INSTALMENT_std', 'NUM_INSTALMENT_VERSION_sum', 'NUM_INSTALMENT_VERSION_mean', 'NUM_INSTALMENT_VERSION_var', 'DAYS_INSTALMENT_DAYS_ENTRY_PAYMENT_DIFF_min', 'DAYS_INSTALMENT_DAYS_ENTRY_PAYMENT_DIFF_sum', 'DAYS_INSTALMENT_DAYS_ENTRY_PAYMENT_DIFF_mean', 'DAYS_INSTALMENT_DAYS_ENTRY_PAYMENT_DIFF_var', 'DAYS_INSTALMENT_DAYS_ENTRY_PAYMENT_DIFF_std', 'AMT_INSTALMENT_AMT_PAYMENT_DIFF_max', 'AMT_INSTALMENT_AMT_PAYMENT_DIFF_sum', 'AMT_INSTALMENT_AMT_PAYMENT_DIFF_mean', 'PREV_NON_CASH_TOTAL_DEBT_TO_AMT_CREDIT_min', 'PREV_NON_CASH_TOTAL_DEBT_TO_AMT_CREDIT_max', 'PREV_NON_CASH_TOTAL_DEBT_TO_AMT_CREDIT_sum', 'PREV_NON_CASH_TOTAL_DEBT_TO_AMT_CREDIT_mean', 'PREV_NON_CASH_TOTAL_DEBT_TO_AMT_CREDIT_var', 'PREV_NON_CASH_TOTAL_DEBT_TO_AMT_CREDIT_std', 'PREV_CASH_TOTAL_DEBT_TO_AMT_CREDIT_min', 'PREV_CASH_TOTAL_DEBT_TO_AMT_CREDIT_max', 'PREV_CASH_TOTAL_DEBT_TO_AMT_CREDIT_sum', 'PREV_CASH_TOTAL_DEBT_TO_AMT_CREDIT_mean', 'PREV_CASH_TOTAL_DEBT_TO_AMT_CREDIT_var', 'DAYS_DECISION_min', 'DAYS_DECISION_max', 'DAYS_DECISION_mean', 'DAYS_DECISION_var']
print(predictors)

**Saving Features to csv file**

In [ ]:
train_final.to_csv('../input/train_final.csv')
test_final.to_csv('../input/test_final.csv')